In [1]:
import os
import sys
from datasets import load_dataset
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


/Users/roshanjeyakumar/elve/myBPE/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("stanfordnlp/imdb")
train_data = ds['train']
test_data = ds['test']
X_train = train_data['text']
y_train = train_data['label']
X_test = test_data['text']
y_test = test_data['label']


In [3]:
Tokenizer = Tokenizer(num_words=100000)
Tokenizer.fit_on_texts(X_train)
maxlen = max([len(x.split()) for x in X_train])
X_train = Tokenizer.texts_to_sequences(X_train)
X_test = Tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)


In [4]:
def embedding_model():
    vocab_size = min(Tokenizer.num_words,len(Tokenizer.word_index) + 1)
    embedding_dim = 128
    model = keras.Sequential([
        keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=512),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    return model


embedding_model = embedding_model()
embedding_model.compile(optimizer='adam',
                          loss='binary_crossentropy',
                          metrics=['accuracy'])
X_train_tensor = tf.convert_to_tensor(X_train)
y_train_tensor = tf.convert_to_tensor(y_train)
X_test_tensor = tf.convert_to_tensor(X_test)
y_test_tensor = tf.convert_to_tensor(y_test)
embedding_model.fit(X_train_tensor, y_train_tensor, epochs=10, batch_size=512, validation_data=(X_test_tensor, y_test_tensor), verbose=1)
embedding_model.save('imdb_embedding_model.h5')
print('accuracy:', embedding_model.evaluate(X_test, y_test, verbose=0)[1])


Epoch 1/10


/Users/roshanjeyakumar/elve/myBPE/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


49/49 ━━━━━━━━━━━━━━━━━━━━ 16s 313ms/step - accuracy: 0.5013 - loss: 0.6950 - val_accuracy: 0.5002 - val_loss: 0.6935
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 15s 306ms/step - accuracy: 0.5034 - loss: 0.6940 - val_accuracy: 0.5006 - val_loss: 0.6934
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 15s 301ms/step - accuracy: 0.5279 - loss: 0.6914 - val_accuracy: 0.5719 - val_loss: 0.6907
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 15s 299ms/step - accuracy: 0.5350 - loss: 0.6913 - val_accuracy: 0.5000 - val_loss: 0.6905
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 15s 301ms/step - accuracy: 0.5210 - loss: 0.6909 - val_accuracy: 0.5926 - val_loss: 0.6882
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 15s 300ms/step - accuracy: 0.5582 - loss: 0.6873 - val_accuracy: 0.5001 - val_loss: 0.6913
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 15s 300ms/step - accuracy: 0.5505 - loss: 0.6861 - val_accuracy: 0.6498 - val_loss: 0.6803
Epoch 8/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 15s 300ms/step - accuracy: 0.6108 - loss: 0.6779 - val_accuracy: 0.503

ValueError: Unrecognized data type: x=[[   0    0    0 ...   29  117  171]
 [   0    0    0 ...   49 5081  467]
 [   0    0    0 ...  335  523  486]
 ...
 [   0    0    0 ...    4    1  169]
 [   0    0    0 ...  596    2 2171]
 [   0    0    0 ...  287 3294   43]] (of type <class 'numpy.ndarray'>)

In [6]:
main()

TypeError: list indices must be integers or slices, not str